# Manipulando o banco de dados

Link do Kagglehub:
<a href="https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information/data?select=song_lyrics.csv">www.kaggle.com/genius-song-lyrics-with-language-information</a>

In [2]:
import pandas as pd
import os

## Tamanho original do Dataset

In [3]:
arq_tamanho = os.stat('data/song_lyrics.csv')
print(f'Tamanho: {arq_tamanho.st_size/1000000000:.2f} GB')

Tamanho: 9.07 GB


Por causa do tamanho, vamos fazer uma filtragem dos dados para poder pegar somente os gêneros musicais desejados e diminuir o peso do nosso Dataset.

## Verificando a estrutura do Dataset e seus gêneros

In [4]:
df = pd.read_csv('data/song_lyrics.csv')
df.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [5]:
print(df['tag'].unique())

['rap' 'rb' 'rock' 'pop' 'misc' 'country']


## Filtragem do Dataframe

Como verificamos, o Dataset possui somente 6 gêneros músicais, sendo alguns deles muito mais populares que outros. 

Para uma versão inicial vamos utilizar somente os gêneros: pop e rock.

In [6]:
generos_desejados = ['pop', 'rock']

df_filtrado = df[df['tag'].isin(generos_desejados) & (df['year'] >= 1960)]

print(f'Tamanho do dataframe original: {df.shape[0]}')
print(f'Tamanho do dataframe filtrado: {df_filtrado.shape[0]}')
df_filtrado.head()

Tamanho do dataframe original: 5134856
Tamanho do dataframe filtrado: 2917784


,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
229,Subterranean Homesick Blues,rock,Bob Dylan,1965,180196,{},"[Verse 1]\nJohnny's in the basement, mixin' up...",224,en,en,en
239,Wordy Rappinghood,pop,Tom Tom Club,1981,26499,{},[Chorus]\nWhat are words worth?\nWhat are word...,242,en,en,en
389,Horchata,pop,Vampire Weekend,2009,102550,{},"[Verse 1]\nIn December, drinking horchata\nI'd...",384,en,en,en
516,Heartless,pop,Kanye West,2008,1175109,{},"[Chorus]\nIn the night, I hear 'em talk\nThe c...",526,en,en,en
557,Flashing Lights,pop,Kanye West,2007,1078113,{Dwele},[Intro: Connie Mitchell]\nFlashing lights (Lig...,523,en,en,en


In [7]:
df_final = df_filtrado.loc[:, ['title', 'tag', 'artist', 'views', 'features', 'lyrics', 'language']].copy()
df_final.head()

,title,tag,artist,views,features,lyrics,language
229,Subterranean Homesick Blues,rock,Bob Dylan,180196,{},"[Verse 1]\nJohnny's in the basement, mixin' up...",en
239,Wordy Rappinghood,pop,Tom Tom Club,26499,{},[Chorus]\nWhat are words worth?\nWhat are word...,en
389,Horchata,pop,Vampire Weekend,102550,{},"[Verse 1]\nIn December, drinking horchata\nI'd...",en
516,Heartless,pop,Kanye West,1175109,{},"[Chorus]\nIn the night, I hear 'em talk\nThe c...",en
557,Flashing Lights,pop,Kanye West,1078113,{Dwele},[Intro: Connie Mitchell]\nFlashing lights (Lig...,en


## Salvando o banco de dados em um arquivo

In [8]:
# DEIXAR COMENTADO
# df_final.to_csv('data/songs_filtrado.csv', index=False, encoding='utf-8')

## Refatoração 
*24 de maio*

Devido ao tamanho do banco de dados anterior, a criação de um BERT com o novo banco de dados (`songs_filtrado.csv`) tinha uma expectativa de demora de aproximadamente 67 horas (para mais, no caso do print: 138 horas).

<div align="center">
    <img src="img/BERT_timeExpected.png" />
</div>

Por conta disso, vamos diminuir um pouco mais a amostra de dados.

In [9]:
print(df_final.shape)
df_final.head()

(2917784, 7)


,title,tag,artist,views,features,lyrics,language
229,Subterranean Homesick Blues,rock,Bob Dylan,180196,{},"[Verse 1]\nJohnny's in the basement, mixin' up...",en
239,Wordy Rappinghood,pop,Tom Tom Club,26499,{},[Chorus]\nWhat are words worth?\nWhat are word...,en
389,Horchata,pop,Vampire Weekend,102550,{},"[Verse 1]\nIn December, drinking horchata\nI'd...",en
516,Heartless,pop,Kanye West,1175109,{},"[Chorus]\nIn the night, I hear 'em talk\nThe c...",en
557,Flashing Lights,pop,Kanye West,1078113,{Dwele},[Intro: Connie Mitchell]\nFlashing lights (Lig...,en


Com um número de amostras relativamente grande, vamos tentar buscar alguma forma de diminuir o número de amostras mas ainda sim buscando melhorar o resultado de nossos embeddings.

A forma mais apropriada de diminuir as amostras, mas esperar um resultado melhor para o usuário, seria reduzir o banco de dados com base no número de visualizações das músicas, pois não seria interessante gastar espaço e tempo de treinamento com músicas pouco, ou muito pouco, conhecidas.

In [10]:
df_filtrado = df[df['views'] > 5500]

df_final = df_filtrado.loc[:, ['title', 'tag', 'artist', 'views', 'features', 'lyrics', 'language']].copy()
print(df_final.shape)
df_final.sort_values(by=['views']).head()

(287261, 7)


,title,tag,artist,views,features,lyrics,language
403902,Ela Partiu,pop,Tim Maia,5501,{},"[Verso]\nEla partiu, partiu e nunca mais volto...",pt
4411052,Sorry Not Sorry,pop,ITZY,5501,{},"[있지 ""Sorry Not Sorry"" 가사]\n\n[Verse 1: Yuna, R...",NaN
503655,The Lost Child,rock,Anathema,5501,{},Adrift on a silent sea\nThe cold night surroun...,en
2563479,Como Le Hago,pop,Ezio Oliva,5501,{},"Letra de ""Como Le Hago ""\n\nSabes muy bien que...",es
2902486,힙합 꼰대 Hip-Hop Ggondae,rap,Dok2,5501,"{""Deepflow (딥플로우)"",""더블케이 (Double K)"",""Snacky C...",[Hook: Dok2]\n우린 힙합만을 고집하는 힙합 꼰대\nRap에 리을도 모르는...,ko


Considerando que, após o corte, começaram a aparecer músicas com um maior reconhecimento, podemos parar por aqui, agora com a amostra muito menor do que a anterior, porém aindad com aproximadamente 300 mil linhas de dados.

In [11]:
df_final = df_final.loc[:, ['title', 'tag', 'artist', 'features', 'lyrics', 'language']]

# Salvando o novo banco de dados
df_final.to_csv('data/songs_filtrado.csv', index=False, encoding='utf-8')